In [4]:
import os
import numpy as np
from PIL import Image, ImageDraw
import shutil
import csv

# Define parameters
image_size = (512, 512)  # Size of the images
num_samples = 200  # Number of samples for each shape
max_translation = 20  # Maximum translation in pixels
max_rotation = 10  # Maximum rotation in degrees
max_shear = 5  # Maximum shear in degrees
max_scale = 1.1  # Maximum scale factor

# Create a function to generate star images
def generate_star_images(num_samples, output_dir="Dataset/synthetic_shape_dataset", num_images=1):
    # delete all files and subdirectories in the output directory
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Create a csv file to store the affine parameters (training and validation sets)
    affine_parameters_path_train = os.path.join(output_dir, "dataset_shape_synth_train.csv")
    with open(affine_parameters_path_train, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M1", "M2", "M3", "M4", "M5", "M6"])
    affine_parameters_path_test = os.path.join(output_dir, "dataset_shape_synth_test.csv")
    with open(affine_parameters_path_test, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M1", "M2", "M3", "M4", "M5", "M6"])

    # save 10% of the images for testing
    num_test = int(num_samples * 0.1)
    num_train = num_samples - num_test
    print(f"Generating {num_images*num_train} training images and {num_images*num_test} testing images")
    print(f"Location of the images: {output_dir}")
    print(f"Saving affine parameters to {affine_parameters_path_train} and {affine_parameters_path_test}")

    # Generate images
    for i in range(num_samples):
        for j in range(num_images):
            # Create a fixed-size image
            image = Image.new("L", image_size, 0)
            draw = ImageDraw.Draw(image)
            
            # Draw a star centered in the image
            center = (image_size[0] // 2, image_size[1] // 2)
            size = np.random.randint(150, min(image_size) // 2)
            points = []
            for _ in range(10):
                angle = np.random.uniform(0, 2 * np.pi)
                x = int(center[0] + size * np.cos(angle))
                y = int(center[1] + size * np.sin(angle))
                points.append((x, y))
            draw.polygon(points, fill=np.random.randint(100, 256))
            
            # Generate random affine transformations
            translation = (
                np.random.randint(-max_translation, max_translation + 1),
                np.random.randint(-max_translation, max_translation + 1),
            )
            rotation = np.random.uniform(-max_rotation, max_rotation)
            shear = np.random.uniform(-max_shear, max_shear)
            scale = np.random.uniform(1.0, max_scale)
            
            # Apply affine transformations
            affine_matrix = [
                scale * np.cos(np.radians(rotation + shear)), -scale * np.sin(np.radians(rotation + shear)), translation[0],
                scale * np.sin(np.radians(rotation)), scale * np.cos(np.radians(rotation)), translation[1]
            ]
            transformed_image = image.transform(image_size, Image.AFFINE, affine_matrix)
            
            # Save original image
            original_image_path = os.path.join(output_dir, f"star_{i+1}_original.png")
            image.save(original_image_path)
            
            # add noise to the transformed image and save it
            transformed_image = transformed_image.point(lambda x: x + np.random.randint(-10, 10))
            transformed_image_path = os.path.join(output_dir, f"star_{i+1}_transformed_{j}.png")
            transformed_image.save(transformed_image_path)
            
            M = np.array(affine_matrix).flatten()
            # Save affine parameters to a csv file (one line per image pair)
            if i < num_test:
                with open(affine_parameters_path_test, "a", newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([original_image_path, transformed_image_path, M[0], M[1], M[2], M[3], M[4], M[5]])
            else:
                with open(affine_parameters_path_train, "a", newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([original_image_path, transformed_image_path, M[0], M[1], M[2], M[3], M[4], M[5]])
    print("Done!")
    print("")

# Generate star images
output_dir = "Dataset/synthetic_shape_dataset"  # Output directory
generate_star_images(num_samples, output_dir=output_dir, num_images=1)

# Generate star images with 2 transformations
output_dir = "Dataset/synthetic_shape_dataset_multiple"  # Output directory
generate_star_images(num_samples, output_dir=output_dir, num_images=2)

Generating 180 training images and 20 testing images
Location of the images: Dataset/synthetic_shape_dataset
Saving affine parameters to Dataset/synthetic_shape_dataset/dataset_shape_synth_train.csv and Dataset/synthetic_shape_dataset/dataset_shape_synth_test.csv
Done!

Generating 360 training images and 40 testing images
Location of the images: Dataset/synthetic_shape_dataset_multiple
Saving affine parameters to Dataset/synthetic_shape_dataset_multiple/dataset_shape_synth_train.csv and Dataset/synthetic_shape_dataset_multiple/dataset_shape_synth_test.csv
Done!



In [ ]:
# grab images in the directory
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

# grab images in the directory
images = glob.glob('Dataset/synthetic_shape_dataset/*.png')
len(images)

# sort images
images.sort()
for i in range(len(images)):
    print(images[i])


Dataset/synthetic_shape_dataset/star_100_original.png
Dataset/synthetic_shape_dataset/star_100_transformed.png
Dataset/synthetic_shape_dataset/star_101_original.png
Dataset/synthetic_shape_dataset/star_101_transformed.png
Dataset/synthetic_shape_dataset/star_102_original.png
Dataset/synthetic_shape_dataset/star_102_transformed.png
Dataset/synthetic_shape_dataset/star_103_original.png
Dataset/synthetic_shape_dataset/star_103_transformed.png
Dataset/synthetic_shape_dataset/star_104_original.png
Dataset/synthetic_shape_dataset/star_104_transformed.png
Dataset/synthetic_shape_dataset/star_105_original.png
Dataset/synthetic_shape_dataset/star_105_transformed.png
Dataset/synthetic_shape_dataset/star_106_original.png
Dataset/synthetic_shape_dataset/star_106_transformed.png
Dataset/synthetic_shape_dataset/star_107_original.png
Dataset/synthetic_shape_dataset/star_107_transformed.png
Dataset/synthetic_shape_dataset/star_108_original.png
Dataset/synthetic_shape_dataset/star_108_transformed.png
D

In [ ]:
# use SIFT to detect and extract features from the image 
# and plot the images in pairs with the keypoints overlaid
# sift = cv2.SIFT_create()

# for i in range(0, 100, 2):
#     fig, ax = plt.subplots(1, 2, figsize=(20, 10))
#     img1 = cv2.imread(images[i], 0)
#     img2 = cv2.imread(images[i+1], 0)
#     kp, des = sift.detectAndCompute(img1, None)
#     img = cv2.drawKeypoints(img1, kp, None)
#     ax[0].imshow(img)
#     ax[0].axis('off')
#     ax[0].set_title(f'#{int(i/2+1)} Original Image')

#     kp, des = sift.detectAndCompute(img2, None)
#     img = cv2.drawKeypoints(img2, kp, None)
#     ax[1].imshow(img)
#     ax[1].axis('off')
#     ax[1].set_title(f'Transformed Image')
#     os.makedirs('Dataset/synthetic_shape_dataset/plot', exist_ok=True)
#     plt.savefig(f'Dataset/synthetic_shape_dataset/plot/{int(i/2+1)}.png')
#     plt.close()

